In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df = pd.read_csv('./datasets/cleaned_data/texas_shelters.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3,9,11,18,19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df.head()

,Unnamed: 0,id,breed,color,dob,sex,date_in,age_in,intact_in,location,intake_type,intake_subtype,condition,date_out,age_out,intact_out,outcome,age,primary_color,secondary_color
0,0,A047759,Dachshund,Tricolor,1.080864e+09,1.0,1.396454e+09,10.0,0.0,Austin (TX),surrender,NaN,wnl,1.396884e+09,10.0,0.0,transfer,Senior,Tricolor,Tricolor
1,1,A134067,Shetland Sheepdog,Brown/White,8.769600e+08,1.0,1.384593e+09,16.0,0.0,12034 Research Blvd in Austin (TX),public_assist,NaN,med_attn,1.384603e+09,16.0,0.0,owner_return,Senior,Brown,White
2,2,A141142,Labrador Retriever/Pit Bull,Black/White,8.966592e+08,0.0,1.384613e+09,15.0,0.0,Austin (TX),stray,NaN,aged,1.384688e+09,15.0,0.0,owner_return,Senior,Black,White
3,3,A163459,Miniature Schnauzer Mix,Black/Gray,9.402912e+08,0.0,1.415978e+09,15.0,1.0,Ih 35 And 41St St in Austin (TX),stray,NaN,wnl,1.415993e+09,15.0,1.0,owner_return,Senior,Black,Gray
4,4,A165752,Lhasa Apso Mix,Brown/White,9.349344e+08,1.0,1.410780e+09,15.0,0.0,Gatlin Gun Rd And Brodie in Austin (TX),stray,NaN,wnl,1.410799e+09,15.0,0.0,owner_return,Senior,Brown,White


In [4]:
df['breed'].nunique()

2588

In [5]:
# lowercase and strip extra white space for breed names
df['breed'] = df['breed'].str.lower().str.strip()

# remove 'mix' at the end of the breed name
df['breed'] = df['breed'].str.replace(' mix', '')

# expand pitbull to pit bull for consistency
df['breed'].replace('pitbull', 'pit bull', inplace=True)

In [6]:
df[df['breed'].str.contains('/') == False]['breed'].nunique()

285

In [7]:
df[df['breed'].str.contains('/')].head()

,Unnamed: 0,id,breed,color,dob,sex,date_in,age_in,intact_in,location,intake_type,intake_subtype,condition,date_out,age_out,intact_out,outcome,age,primary_color,secondary_color
2,2,A141142,labrador retriever/pit bull,Black/White,896659200.0,0.0,1.384613e+09,15.0,0.0,Austin (TX),stray,NaN,aged,1.384688e+09,15.0,0.0,owner_return,Senior,Black,White
10,10,A212672,german shepherd/labrador retriever,Tan/Black,952214400.0,0.0,1.385400e+09,13.0,1.0,14611 Wells Port Dr in Austin (TX),stray,NaN,wnl,1.386340e+09,13.0,1.0,owner_return,Senior,Tan,Black
16,16,A226069,labrador retriever/beagle,Sable/White,961200000.0,1.0,1.444135e+09,15.0,0.0,Wheless Ln And Berkman Dr in Austin (TX),stray,NaN,wnl,1.446131e+09,15.0,0.0,adopted,Senior,Sable,White
17,17,A228887,catahoula/labrador retriever,Blue Merle/Tan,988329600.0,0.0,1.384016e+09,12.0,0.0,Lincolnshire in Austin (TX),stray,NaN,wnl,1.384019e+09,12.0,0.0,owner_return,Senior,Blue,Merle
19,19,A233812,rottweiler/chow chow,Brown/Black,991958400.0,1.0,1.410014e+09,13.0,0.0,12317 Wycliff Ln in Austin (TX),stray,NaN,aged,1.410030e+09,13.0,0.0,owner_return,Senior,Brown,Black


In [8]:
df[df['breed'].str.contains('/')]['breed'].nunique()

2023

In [9]:
df['breed'].value_counts(ascending=True)[df['breed'].value_counts(ascending=True) == 1]

welsh spr span                                 1
blue lacy/cardigan welsh corgi                 1
chinese sharpei/chow chow                      1
maltese/cocker spaniel                         1
english springer spaniel/labrador retriever    1
                                              ..
schipperke/catahoula                           1
german shorthair pointer/bluetick hound        1
bulldog/chinese sharpei                        1
st. bernard rough coat/beagle                  1
weimaraner/american bulldog                    1
Name: breed, Length: 917, dtype: int64

In [10]:
# regex to replace breed names to remain consistent
df['breed'] = df['breed'].map(lambda x: re.sub('labrador retr\Z', 'labrador retriever', x))
df['breed'] = df['breed'].map(lambda x: re.sub('germ\sshepherd', 'german shepherd', x))
df['breed'] = df['breed'].map(lambda x: re.sub('^staffordshire$', 'american staffordshire terrier', x))
df['breed'] = df['breed'].map(lambda x: re.sub('chihuahua shorthair', 'chihuahua', x))
df['breed'] = df['breed'].map(lambda x: re.sub('chihuahua longhair', 'chihuahua', x))
df['breed'] = df['breed'].map(lambda x: re.sub('chihuahua sh', 'chihuahua', x))
df['breed'] = df['breed'].map(lambda x: re.sub('chihuahua lh', 'chihuahua', x))
df['breed'] = df['breed'].map(lambda x: re.sub('airedale terr\Z', 'airedale terrier', x))
df['breed'] = df['breed'].map(lambda x: re.sub('amer ', 'american', x))
df['breed'] = df['breed'].map(lambda x: re.sub('alask ', 'alaskan', x))
df['breed'] = df['breed'].map(lambda x: re.sub('am pit bull ter', 'pit bull', x))
df['breed'] = df['breed'].map(lambda x: re.sub('aust ', 'australian', x))
df['breed'] = df['breed'].map(lambda x: re.sub('american bulldog', 'bulldog', x))
df['breed'] = df['breed'].map(lambda x: re.sub('english bulldog', 'bulldog', x))
df['breed'] = df['breed'].map(lambda x: re.sub('black/tan hound', 'black and tan coonhound', x))
# start of nate edit
df['breed'] = df['breed'].map(lambda x: re.sub('yorkshire terr\Z', 'yorkshire terrier', x))
df['breed'] = df['breed'].map(lambda x: re.sub('australiancattle dog', 'australian cattle dog', x))
df['breed'] = df['breed'].map(lambda x: re.sub('anatol shepherd', 'anatolian sheepdog', x))
df['breed'] = df['breed'].map(lambda x: re.sub('doberman pinsch\Z', 'doberman pinscher', x))
df['breed'] = df['breed'].map(lambda x: re.sub('poodle min\Z', 'miniature poodle', x))

In [28]:
df['breed'] = df['breed'].map(lambda x: re.sub('poodle min\Z', 'miniature poodle', x))

In [29]:
df[df['breed'].str.contains('poodle min')]

,Unnamed: 0,id,breed,color,dob,sex,date_in,age_in,intact_in,location,...,date_out,age_out,intact_out,outcome,age,primary_color,secondary_color,breed_1,breed_2,pure


In [13]:
# read in breed info for all dogs, convert it to a dictionary
all_dogs = pd.read_csv('./datasets/all_breeds_info.csv')
pure_dogs = pd.read_csv('./datasets/dog_intelligence.csv')
master_list = set(list(all_dogs['breed']) + list(pure_dogs['Breed'].str.lower()))
all_dogs = all_dogs.to_dict(orient='records')

In [14]:
df.head()

,Unnamed: 0,id,breed,color,dob,sex,date_in,age_in,intact_in,location,intake_type,intake_subtype,condition,date_out,age_out,intact_out,outcome,age,primary_color,secondary_color
0,0,A047759,dachshund,Tricolor,1.080864e+09,1.0,1.396454e+09,10.0,0.0,Austin (TX),surrender,NaN,wnl,1.396884e+09,10.0,0.0,transfer,Senior,Tricolor,Tricolor
1,1,A134067,shetland sheepdog,Brown/White,8.769600e+08,1.0,1.384593e+09,16.0,0.0,12034 Research Blvd in Austin (TX),public_assist,NaN,med_attn,1.384603e+09,16.0,0.0,owner_return,Senior,Brown,White
2,2,A141142,labrador retriever/pit bull,Black/White,8.966592e+08,0.0,1.384613e+09,15.0,0.0,Austin (TX),stray,NaN,aged,1.384688e+09,15.0,0.0,owner_return,Senior,Black,White
3,3,A163459,miniature schnauzer,Black/Gray,9.402912e+08,0.0,1.415978e+09,15.0,1.0,Ih 35 And 41St St in Austin (TX),stray,NaN,wnl,1.415993e+09,15.0,1.0,owner_return,Senior,Black,Gray
4,4,A165752,lhasa apso,Brown/White,9.349344e+08,1.0,1.410780e+09,15.0,0.0,Gatlin Gun Rd And Brodie in Austin (TX),stray,NaN,wnl,1.410799e+09,15.0,0.0,owner_return,Senior,Brown,White


In [15]:
df['breed_1'] = np.nan
df['breed_2'] = np.nan
df['pure'] = np.nan

In [16]:
def breed_check(x):
    if x in master_list:
        return x

df['breed_1'] = df['breed'].apply(breed_check)
df['breed_2'] = df['breed'].apply(breed_check)
df['pure'] = np.where(df['breed_1'] == df['breed_2'], 1, np.nan)


In [17]:
df['breed_1'].isna().sum()

30438

In [18]:
df['pure'].isna().sum()

30438

In [19]:
df[df['breed_1'].isna()]['breed'].value_counts()

rat terrier                                  2152
catahoula                                    1346
alaskan husky                                 820
black mouth cur                               781
australian kelpie                             680
                                             ... 
labrador retriever/st. bernard rough coat       1
boxer/great pyrenees                            1
west highland/maltese                           1
cavalier span/pug                               1
welsh spr span                                  1
Name: breed, Length: 2101, dtype: int64

In [20]:
# loops over all breeds in dataframe
df_mini = df[(df['breed_1'].isna()) & (df['breed'].str.contains('/'))].copy()
for i, dog in df_mini['breed'].items():

    # splits the dogs on the '/'
    breeds = dog.split('/')

    # checks if any items of breeds is in the master list of dog breeds
    if not set(breeds).isdisjoint(master_list):
        
        # checks both position combinations
        if (breeds[0] in master_list) and (breeds[1] in master_list):
            df.loc[i, 'breed_1'] = breeds[0]
            df.loc[i, 'breed_2'] = breeds[1]

In [21]:
df['breed_1'].isna().sum()

16388

In [22]:
for i, v in df[df['breed'].str.contains('/')]['breed'].items():
    v = v.split('/')
    if v[0] in master_list and not v[1] in master_list:
        df.loc[i, 'breed_1'] = v[0]
        df.loc[i, 'breed_2'] = 'mix'
    elif not v[0] in master_list and v[1] in master_list:
        df.loc[i, 'breed_1'] = v[1]
        df.loc[i, 'breed_2'] = 'mix'

In [23]:
'poodle' in master_list

True

In [24]:
df['breed_1'].isna().sum()

12605

In [25]:
set(df[df['breed_1'].isna()]['breed']).difference(master_list)

{'akbash',
 'alaskan husky',
 'alaskan husky/catahoula',
 'alaskan husky/collie smooth',
 'alaskan klee kai',
 'alaskanklee kai',
 'alaskanmalamute',
 'american staff',
 'americanbulldog',
 'americaneskimo',
 'americanfoxhound',
 'americanwater span',
 'australian kelpie',
 'australian kelpie/alaskan husky',
 'australian kelpie/blue lacy',
 'australian kelpie/catahoula',
 'australian kelpie/chinese sharpei',
 'australian kelpie/rat terrier',
 'australian kelpie/staffordshire',
 'australiankelpie',
 'australianshepherd',
 'australianterrier',
 'bedlington terr',
 'belg malinois',
 'belg sheepdog',
 'belg tervuren',
 'bernese hound',
 'bernese mtn dog',
 'black mouth cur',
 'black mouth cur/blue lacy',
 'black mouth cur/catahoula',
 'black mouth cur/chinese sharpei',
 'black mouth cur/rhod ridgeback',
 'black mouth cur/staffordshire',
 'blue lacy',
 'blue lacy/australian kelpie',
 'blue lacy/black mouth cur',
 'blue lacy/collie smooth',
 'blue lacy/staffordshire',
 'bluetick hound',
 'bl

In [26]:
df[df['breed_1'].isna()]['breed'].value_counts()[:20]

rat terrier            2152
catahoula              1346
alaskan husky           820
black mouth cur         781
australian kelpie       680
chinese sharpei         352
blue lacy               337
queensland heeler       279
collie smooth           232
rhod ridgeback          221
carolina dog            216
dachshund longhair      204
poodle min              186
redbone hound           180
manchester terrier      179
schnauzer min           179
flat coat retriever     177
australianshepherd      173
americanbulldog         155
dachshund wirehair      151
Name: breed, dtype: int64

In [27]:
df.shape

(132618, 23)

In [31]:
df.to_csv('./datasets/cleaned_data/texas_shelters_breeds_cleaned.csv', index=False)